In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.preprocessing import MinMaxScaler
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings
warnings.filterwarnings('ignore')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/playground-series-s3e20/sample_submission.csv
/kaggle/input/playground-series-s3e20/train.csv
/kaggle/input/playground-series-s3e20/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s3e20/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e20/test.csv')

In [3]:
train

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-0.510_29.290_2019_00,-0.510,29.290,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,...,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994
1,ID_-0.510_29.290_2019_01,-0.510,29.290,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,...,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779,4.025176
2,ID_-0.510_29.290_2019_02,-0.510,29.290,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,...,4216.986492,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296,4.231381
3,ID_-0.510_29.290_2019_03,-0.510,29.290,2019,3,NaN,NaN,NaN,NaN,NaN,...,5228.507736,51064.547339,4180.973322,15.386899,0.262043,-11.293399,24.380357,-132.665828,28.829155,4.305286
4,ID_-0.510_29.290_2019_04,-0.510,29.290,2019,4,-0.000079,0.676296,-0.000048,0.121164,4.121269,...,3980.598120,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612,4.347317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79018,ID_-3.299_30.301_2021_48,-3.299,30.301,2021,48,0.000284,1.195643,0.000340,0.191313,72.820518,...,5459.185355,60657.101913,4590.879504,20.245954,0.304797,-35.140368,40.113533,-129.935508,32.095214,29.404171
79019,ID_-3.299_30.301_2021_49,-3.299,30.301,2021,49,0.000083,1.130868,0.000063,0.177222,-12.856753,...,5606.449457,60168.191528,4659.130378,6.104610,0.314015,4.667058,47.528435,-134.252871,30.771469,29.186497
79020,ID_-3.299_30.301_2021_50,-3.299,30.301,2021,50,NaN,NaN,NaN,NaN,NaN,...,6222.646776,56596.027209,5222.646823,14.817885,0.288058,-0.340922,35.328098,-134.731723,30.716166,29.131205
79021,ID_-3.299_30.301_2021_51,-3.299,30.301,2021,51,-0.000034,0.879397,-0.000028,0.184209,-100.344827,...,7896.456885,46533.348194,6946.858022,32.594768,0.274047,8.427699,48.295652,-139.447849,29.112868,28.125792


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 76 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   ID_LAT_LON_YEAR_WEEK                                      79023 non-null  object 
 1   latitude                                                  79023 non-null  float64
 2   longitude                                                 79023 non-null  float64
 3   year                                                      79023 non-null  int64  
 4   week_no                                                   79023 non-null  int64  
 5   SulphurDioxide_SO2_column_number_density                  64414 non-null  float64
 6   SulphurDioxide_SO2_column_number_density_amf              64414 non-null  float64
 7   SulphurDioxide_SO2_slant_column_number_density            64414 non-null  float64
 8   SulphurDioxide_c

In [5]:
train_df = train.drop(columns=['ID_LAT_LON_YEAR_WEEK','UvAerosolLayerHeight_aerosol_height','UvAerosolLayerHeight_aerosol_pressure','UvAerosolLayerHeight_aerosol_optical_depth','UvAerosolLayerHeight_sensor_zenith_angle','UvAerosolLayerHeight_sensor_azimuth_angle','UvAerosolLayerHeight_solar_azimuth_angle','UvAerosolLayerHeight_solar_zenith_angle'])
test_df = test.drop(columns=['ID_LAT_LON_YEAR_WEEK','UvAerosolLayerHeight_aerosol_height','UvAerosolLayerHeight_aerosol_pressure','UvAerosolLayerHeight_aerosol_optical_depth','UvAerosolLayerHeight_sensor_zenith_angle','UvAerosolLayerHeight_sensor_azimuth_angle','UvAerosolLayerHeight_solar_azimuth_angle','UvAerosolLayerHeight_solar_zenith_angle'])

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 68 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   latitude                                                  79023 non-null  float64
 1   longitude                                                 79023 non-null  float64
 2   year                                                      79023 non-null  int64  
 3   week_no                                                   79023 non-null  int64  
 4   SulphurDioxide_SO2_column_number_density                  64414 non-null  float64
 5   SulphurDioxide_SO2_column_number_density_amf              64414 non-null  float64
 6   SulphurDioxide_SO2_slant_column_number_density            64414 non-null  float64
 7   SulphurDioxide_cloud_fraction                             64414 non-null  float64
 8   SulphurDioxide_s

In [7]:
train_df2=train_df.fillna(0)
test_df2=test_df.fillna(0)

In [8]:
#train_df2['year']=train_df2['year'].astype('category')
#test_df2['year']=test_df2['year'].astype('category')

In [9]:
test_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24353 entries, 0 to 24352
Data columns (total 67 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   latitude                                                  24353 non-null  float64
 1   longitude                                                 24353 non-null  float64
 2   year                                                      24353 non-null  int64  
 3   week_no                                                   24353 non-null  int64  
 4   SulphurDioxide_SO2_column_number_density                  24353 non-null  float64
 5   SulphurDioxide_SO2_column_number_density_amf              24353 non-null  float64
 6   SulphurDioxide_SO2_slant_column_number_density            24353 non-null  float64
 7   SulphurDioxide_cloud_fraction                             24353 non-null  float64
 8   SulphurDioxide_s

In [10]:
#train_df2 = pd.get_dummies(train_df2 , columns=['year'],drop_first=True)
#test_df2 = pd.get_dummies(test_df2 , columns=['year'])

In [11]:
test_df2

,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,...,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle
0,-0.510,29.290,2022,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,36022.027344,8472.313477,41047.937500,7472.313477,7.935617,0.240773,-100.113792,33.697044,-133.047546,33.779583
1,-0.510,29.290,2022,1,0.000456,0.691164,0.000316,0.000000,76.239196,15.600607,...,48539.737242,6476.147323,54915.708579,5476.147161,11.448437,0.293119,-30.510319,42.402593,-138.632822,31.012380
2,-0.510,29.290,2022,2,0.000161,0.605107,0.000106,0.079870,-42.055341,39.889060,...,34133.080469,8984.795703,39006.093750,7984.795703,10.753179,0.267130,39.087361,45.936480,-144.784988,26.743361
3,-0.510,29.290,2022,3,0.000350,0.696917,0.000243,0.201028,72.169566,58.862543,...,50854.991076,6014.724059,57646.368368,5014.724115,11.764556,0.304679,-24.465127,42.140419,-135.027891,29.604774
4,-0.510,29.290,2022,4,-0.000317,0.580527,-0.000184,0.204352,76.190865,15.646016,...,46594.685145,6849.280477,52896.541873,5849.280394,13.065317,0.284221,-12.907850,30.122641,-135.500119,26.276807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24348,-3.299,30.301,2022,44,-0.000618,0.745549,-0.000461,0.234492,72.306198,61.114494,...,48839.430415,6260.120033,55483.459980,5260.120056,30.398508,0.180046,-25.528588,45.284576,-116.521412,29.992562
24349,-3.299,30.301,2022,45,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,47042.694849,6678.843299,53589.917383,5678.951521,19.223844,0.177833,-13.380005,43.770351,-122.405759,29.017975
24350,-3.299,30.301,2022,46,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,55337.148173,5336.282475,62646.761340,4336.282491,13.801194,0.219471,-5.072065,33.226455,-124.530639,30.187472
24351,-3.299,30.301,2022,47,0.000071,1.003805,0.000077,0.205077,74.327427,38.215228,...,44813.691428,7188.578533,50728.313991,6188.578464,27.887489,0.247275,-0.668714,45.885617,-129.006797,30.427455


In [12]:
train_df2['Iscovid'] = np.where((train_df2['year'] == 2020) & 
                                (train_df2['week_no'] >= 12) &
                                (train_df2['week_no'] <=26), 1, 0)

In [13]:
test_df2['Iscovid'] = np.where((test_df2['year'] == 2020) & 
                                (test_df2['week_no'] >= 12) &
                                (test_df2['week_no'] <=26), 1, 0)

In [14]:
train_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 69 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   latitude                                                  79023 non-null  float64
 1   longitude                                                 79023 non-null  float64
 2   year                                                      79023 non-null  int64  
 3   week_no                                                   79023 non-null  int64  
 4   SulphurDioxide_SO2_column_number_density                  79023 non-null  float64
 5   SulphurDioxide_SO2_column_number_density_amf              79023 non-null  float64
 6   SulphurDioxide_SO2_slant_column_number_density            79023 non-null  float64
 7   SulphurDioxide_cloud_fraction                             79023 non-null  float64
 8   SulphurDioxide_s

In [15]:
rf = RandomForestRegressor(n_estimators=50,verbose=3,random_state=1)

In [16]:
x=train_df2.drop(columns=['emission'])
y=train_df2['emission']

In [17]:
mm=MinMaxScaler()
xs=x.drop(columns=['latitude','longitude','week_no','Iscovid'])


In [18]:
model = mm.fit(xs)
scaled_data = model.transform(xs)

In [19]:
scaled_data = pd.DataFrame(data=scaled_data,columns=xs.columns)

In [20]:
df=train_df2[['latitude','longitude','week_no','Iscovid']]

In [21]:
scaled_data = pd.concat([scaled_data,df],axis=1)

In [22]:
scaled_data

,year,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,SulphurDioxide_solar_azimuth_angle,SulphurDioxide_solar_zenith_angle,SulphurDioxide_SO2_column_number_density_15km,...,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,latitude,longitude,week_no,Iscovid
0,0.0,0.171182,0.319864,0.160403,0.852234,0.268351,0.767542,0.173175,0.836486,0.394492,...,0.062274,0.369703,0.497952,0.540284,0.095643,0.731142,-0.510,29.290,0,0
1,0.0,0.196027,0.386271,0.175811,0.436631,0.650229,0.590821,0.104361,0.675381,0.406572,...,0.034762,0.348711,0.735505,0.599801,0.053956,0.647920,-0.510,29.290,1,0
2,0.0,0.291193,0.396872,0.248237,0.366730,0.836558,0.798116,0.045124,0.541104,0.449851,...,0.084414,0.340932,0.652718,0.460974,0.071317,0.622754,-0.510,29.290,2,0
3,0.0,0.192069,0.000000,0.173154,0.000000,0.595218,0.000000,1.000000,0.000000,0.402783,...,0.061548,0.355788,0.505332,0.369672,0.135526,0.685422,-0.510,29.290,3,0
4,0.0,0.176884,0.358732,0.163859,0.403884,0.608882,0.536149,0.126392,0.567349,0.394340,...,0.032459,0.320221,0.780669,0.566979,0.077897,0.527922,-0.510,29.290,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79018,1.0,0.246818,0.634213,0.239425,0.637715,0.836640,0.845204,0.104699,0.586390,0.442826,...,0.080984,0.413837,0.373554,0.608230,0.153317,0.763074,-3.299,30.301,48,0
79019,1.0,0.208094,0.599854,0.185505,0.590745,0.552594,0.293399,0.166413,0.727436,0.412013,...,0.024418,0.426353,0.593530,0.720660,0.125184,0.731601,-3.299,30.301,49,0
79020,1.0,0.192069,0.000000,0.173154,0.000000,0.595218,0.000000,1.000000,0.000000,0.402783,...,0.059272,0.391110,0.565856,0.535670,0.122064,0.730286,-3.299,30.301,50,0
79021,1.0,0.185529,0.466464,0.167703,0.614036,0.262546,0.492126,0.176210,0.790588,0.398327,...,0.130379,0.372087,0.614311,0.732293,0.091333,0.692168,-3.299,30.301,51,0


In [23]:
rf.fit(scaled_data,y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 50


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.8s remaining:    0.0s


building tree 2 of 50


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   13.5s remaining:    0.0s


building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 o

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  5.6min finished


RandomForestRegressor(n_estimators=50, random_state=1, verbose=3)

In [24]:
rf.feature_importances_

array([1.47929911e-03, 2.05050085e-04, 3.63021918e-04, 2.32824513e-04,
       3.28516182e-04, 2.37268876e-04, 2.59316560e-04, 4.26087990e-04,
       1.78675476e-04, 2.68854722e-04, 1.51941676e-03, 5.35556518e-04,
       4.91088340e-04, 3.92928403e-04, 2.97904602e-04, 3.58311828e-04,
       1.45709786e-03, 4.02211835e-04, 2.34986572e-04, 2.39349194e-04,
       3.41384034e-04, 1.65448378e-04, 3.24997072e-04, 3.54990821e-04,
       2.70240666e-04, 2.62359575e-04, 2.68453817e-04, 1.66983816e-04,
       2.36924368e-04, 1.34342437e-04, 2.56149863e-04, 4.89819572e-04,
       2.35154163e-04, 2.59109345e-04, 2.06694634e-04, 4.51681845e-04,
       2.78960394e-04, 2.34806613e-04, 7.09045249e-04, 2.70037395e-04,
       5.02595619e-04, 1.46629448e-04, 1.50247134e-03, 9.46302315e-04,
       2.64481005e-03, 1.49400889e-04, 2.92413194e-04, 7.09977207e-04,
       2.34697301e-04, 3.62899187e-04, 1.39518149e-04, 1.20759911e-03,
       4.44766985e-04, 2.25918139e-04, 2.36493417e-04, 2.58973793e-04,
      

In [25]:
sc_data=pd.Series(data=rf.feature_importances_,index=scaled_data.columns)

In [26]:
sc_data.sort_values().tail(7).index

Index(['CarbonMonoxide_CO_column_number_density',
       'Ozone_O3_column_number_density', 'Cloud_surface_albedo', 'Iscovid',
       'week_no', 'latitude', 'longitude'],
      dtype='object')

In [27]:
rf1 = RandomForestRegressor(n_estimators=1000,verbose=3,random_state=1,max_depth=15)

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
x=scaled_data[['CarbonMonoxide_CO_column_number_density',
       'Ozone_O3_column_number_density', 'Cloud_surface_albedo', 'Iscovid',
       'week_no', 'latitude', 'longitude']]

In [30]:
y

0         3.750994
1         4.025176
2         4.231381
3         4.305286
4         4.347317
           ...    
79018    29.404171
79019    29.186497
79020    29.131205
79021    28.125792
79022    27.239302
Name: emission, Length: 79023, dtype: float64

In [31]:
rf1.fit(x,y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 1000


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


building tree 2 of 1000


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building tree 41 of 1000
building tree 42 of 1000
buildin

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  6.4min finished


RandomForestRegressor(max_depth=15, n_estimators=1000, random_state=1,
                      verbose=3)

In [32]:
test_df = test.drop(['ID_LAT_LON_YEAR_WEEK'], axis = 1)

In [33]:
test_df2.sample(5)

,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,Iscovid
14082,-2.056,30.544,2022,19,-0.000145,0.889149,-0.000129,0.000000,-102.405525,6.097099,...,6107.354576,55844.642427,5107.354576,10.685380,0.331209,-41.144213,49.903305,-50.677358,33.953073,0
1125,-0.799,29.201,2022,47,0.000025,0.895772,0.000036,0.095084,-14.095539,41.441904,...,4329.312298,71342.626404,3516.204258,13.108051,0.293225,74.587428,33.737853,-134.438214,28.783998,0
4877,-1.187,31.113,2022,26,0.000123,0.847929,0.000105,0.254347,72.880688,52.532798,...,3106.300047,79260.856692,2151.048561,6.493562,0.242049,-42.103505,35.016102,-40.361114,33.401886,0
8657,-1.529,30.971,2022,33,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5846.191017,58063.041695,4846.191077,23.157984,0.227929,-12.916247,30.131011,-53.457596,26.184728,0
4630,-1.173,30.727,2022,24,0.000078,0.693405,0.000047,0.261842,76.159203,16.000907,...,4388.946767,68522.717956,3388.946767,11.514727,0.293524,11.222577,44.323818,-38.637058,32.902090,0


In [34]:
test_df=test_df2[['CarbonMonoxide_CO_column_number_density',
       'Ozone_O3_column_number_density', 'Cloud_surface_albedo', 'Iscovid',
       'week_no', 'latitude', 'longitude']]

In [35]:
m1=MinMaxScaler()
model1 = m1.fit(test_df)
scaled_test = model1.transform(test_df)

In [36]:
test_df.isnull().sum()

CarbonMonoxide_CO_column_number_density    0
Ozone_O3_column_number_density             0
Cloud_surface_albedo                       0
Iscovid                                    0
week_no                                    0
latitude                                   0
longitude                                  0
dtype: int64

In [39]:
predictions = rf1.predict(test_df)
# # Create a submission file
sub_file = pd.DataFrame({'ID_LAT_LON_YEAR_WEEK': test.ID_LAT_LON_YEAR_WEEK, 'emission': predictions})
sub_file.head()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    1.7s finished


,ID_LAT_LON_YEAR_WEEK,emission
0,ID_-0.510_29.290_2022_00,3.464888
1,ID_-0.510_29.290_2022_01,4.182202
2,ID_-0.510_29.290_2022_02,4.284383
3,ID_-0.510_29.290_2022_03,4.264717
4,ID_-0.510_29.290_2022_04,4.269355


In [40]:
sub_file.to_csv('BaselineSubmission25.csv', index = False) # Download and submit, or submit via Notebook output